# CD3_P1 Fine Tuning

### Import Packages

In [14]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16

### Load Dataset

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

In [16]:
# Define paths to training and validation datasets
data_dir = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\DatasetConv'

# Load Datasets
train_dataset = image_dataset_from_directory(
    data_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    validation_split=0.2,  # 20% for validation
    subset='training',     # Use the 'training' subset
    seed=123
)

val_dataset = image_dataset_from_directory(
    data_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    validation_split=0.2,  # 20% for validation
    subset='validation',   # Use the 'validation' subset
    seed=123
)

Found 24997 files belonging to 2 classes.
Using 19998 files for training.
Found 24997 files belonging to 2 classes.
Using 4999 files for validation.


### Preprocessing & Augmentation

In [17]:
from tensorflow.keras.applications.vgg16 import preprocess_input

In [18]:
# Augmentation layer
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),  # 10% random rotation
    layers.RandomZoom(0.1),      # 10% zoom
    layers.RandomTranslation(0.1, 0.1),  # Random height and width shift
    layers.RandomBrightness(0.2)
])

# Augment the training data
def augment_img(image, label):
    image = data_augmentation(image)  # Apply augmentations
    return image, label

train_dataset = train_dataset.map(augment_img)

In [19]:
# Apply VGG-16 preprocessing
def preprocess_img(image, label):
    image = preprocess_input(image)  # Apply VGG16-specific preprocessing
    return image, label

train_dataset = train_dataset.map(preprocess_img)
val_dataset = val_dataset.map(preprocess_img)

### Load the Model with the Trained Model with the Trained Head

In [20]:
filepath = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\StarredModels\\CD3_P1_head_epoch010_val0.0396.keras'
model = tf.keras.models.load_model(filepath)

### Unfreeze Last VGG Block & Verify

In [21]:
# Unfreeze the last few layers (e.g., last 4 layers)
for layer in model.layers[-4:]:
    layer.trainable = True

# Optionally, print trainable status
for i, layer in enumerate(model.layers):
    print(f"Layer {i}: {layer.name}, Trainable: {layer.trainable}")

Layer 0: vgg16, Trainable: True
Layer 1: global_average_pooling2d_1, Trainable: True
Layer 2: dense_1, Trainable: True
Layer 3: batch_normalization, Trainable: True
Layer 4: dropout, Trainable: True
Layer 5: dense_2, Trainable: True


In [22]:
for layer in model.layers[0].layers:
    if layer.name in ['block5_conv1', 'block5_conv2', 'block5_conv3']:
        layer.trainable = True
    else:
        layer.trainable = False

In [23]:
def print_model_layers(model, indent=0):
    for layer in model.layers:
        print(" " * indent + f"- {layer.name} ({layer.__class__.__name__}), Trainable: {layer.trainable}")
        # If this layer has sublayers (like Functional or Sequential models)
        if hasattr(layer, 'layers'):
            print_model_layers(layer, indent + 2)

print_model_layers(model)

print(model.optimizer.get_config())

- vgg16 (Functional), Trainable: True
  - input_layer_1 (InputLayer), Trainable: False
  - block1_conv1 (Conv2D), Trainable: False
  - block1_conv2 (Conv2D), Trainable: False
  - block1_pool (MaxPooling2D), Trainable: False
  - block2_conv1 (Conv2D), Trainable: False
  - block2_conv2 (Conv2D), Trainable: False
  - block2_pool (MaxPooling2D), Trainable: False
  - block3_conv1 (Conv2D), Trainable: False
  - block3_conv2 (Conv2D), Trainable: False
  - block3_conv3 (Conv2D), Trainable: False
  - block3_pool (MaxPooling2D), Trainable: False
  - block4_conv1 (Conv2D), Trainable: False
  - block4_conv2 (Conv2D), Trainable: False
  - block4_conv3 (Conv2D), Trainable: False
  - block4_pool (MaxPooling2D), Trainable: False
  - block5_conv1 (Conv2D), Trainable: True
  - block5_conv2 (Conv2D), Trainable: True
  - block5_conv3 (Conv2D), Trainable: True
  - block5_pool (MaxPooling2D), Trainable: False
- global_average_pooling2d_1 (GlobalAveragePooling2D), Trainable: True
- dense_1 (Dense), Trainable

### Compile

In [24]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=1e-5)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

### Train & Save Results

In [25]:
# Define the checkpoint path
model_name = 'CD3_P1_best'
checkpoint_path = f"SavedModels/{model_name}_ft_{{epoch:03d}}_val{{val_loss:.4f}}.keras"

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# Create the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_freq='epoch',              # Save every epoch
    save_weights_only=False,
    save_best_only=False,           # Save every time, not just best
    monitor='val_loss',
    verbose=1,
)

# Reduce Learning Rate on Plateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',       # Watch validation loss
    factor=0.1,               # Reduce LR by 10x
    patience=2,               # Wait for 3 epochs with no improvement
    verbose=1,                # Print when LR is reduced
    min_lr=1e-6               # Don't go below this LR
)

# Stop training early if no improvement
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,                 # stop if no improvement for 5 epochs
    restore_best_weights=True,  # load back best weights automatically
    verbose=1
)

In [ ]:
history = model.fit(
    train_dataset,
    epochs=7,
    validation_data=val_dataset,
    callbacks=[checkpoint_callback, reduce_lr]
)

Epoch 1/7
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9714 - loss: 0.0752
Epoch 1: saving model to SavedModels/CD3_P1_best_ft_001_val0.0358.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 3863s 6s/step - accuracy: 0.9714 - loss: 0.0752 - val_accuracy: 0.9886 - val_loss: 0.0358 - learning_rate: 1.0000e-05
Epoch 2/7
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.9765 - loss: 0.0645
Epoch 2: saving model to SavedModels/CD3_P1_best_ft_002_val0.0370.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 6386s 10s/step - accuracy: 0.9765 - loss: 0.0645 - val_accuracy: 0.9894 - val_loss: 0.0370 - learning_rate: 1.0000e-05
Epoch 3/7
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9798 - loss: 0.0554
Epoch 3: saving model to SavedModels/CD3_P1_best_ft_003_val0.0424.keras

Epoch 3: ReduceLROnPlateau reducing learning rate to 1e-06.
625/625 ━━━━━━━━━━━━━━━━━━━━ 3426s 5s/step - accuracy: 0.9798 - loss: 0.0554 - val_accuracy: 0.9888 - val_loss: 0.0424 - learning_rate: 1.0000e-05
Epoch 4/7
625/625 ━━━━━━━━━━━

In [ ]:
import json

model_name = 'CD3_P1_FT'
path = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\Docs_Reports\\RawTrainingData\\'
filepath = f"{path}{model_name}.json"
# filepath = "abc.json"
with open(filepath, 'w') as f:
    json.dump(history.history, f)

### Continue for a few more epochs

In [29]:
history2 = model.fit(
    train_dataset,
    epochs=8,
    validation_data=val_dataset,
    callbacks=[checkpoint_callback]
)

Epoch 1/8
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9886 - loss: 0.0386
Epoch 1: saving model to SavedModels/CD3_P1_best_ft_001_val0.0385.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 3370s 5s/step - accuracy: 0.9886 - loss: 0.0386 - val_accuracy: 0.9904 - val_loss: 0.0385
Epoch 2/8
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9877 - loss: 0.0352
Epoch 2: saving model to SavedModels/CD3_P1_best_ft_002_val0.0385.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 3374s 5s/step - accuracy: 0.9877 - loss: 0.0352 - val_accuracy: 0.9910 - val_loss: 0.0385
Epoch 3/8
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9881 - loss: 0.0327
Epoch 3: saving model to SavedModels/CD3_P1_best_ft_003_val0.0386.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 3356s 5s/step - accuracy: 0.9881 - loss: 0.0327 - val_accuracy: 0.9908 - val_loss: 0.0386
Epoch 4/8
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9889 - loss: 0.0319
Epoch 4: saving model to SavedModels/CD3_P1_best_ft_004_val0.0371.keras
625/625 ━━━━━━━━━━━

In [41]:
import json

model_name = 'CD3_P1_FT_continue'
path = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\Docs_Reports\\RawTrainingData\\'
filepath = f"{path}{model_name}.json"
with open(filepath, 'w') as f:
    json.dump(history2.history, f)